In [1]:
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [105]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

driver.get('https://www.nusabali.com/search?keyword=kependudukan')

count = 0

while (count<15):
    try:
        time.sleep(1)
        driver.execute_script("arguments[0].click();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#main-content > div.wrapper.clearfix > div.col-a.pull-left > section.widget-area-2.pull-right > div > div > div.row > div > button'))))
        count +=1
    except: 
        break

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 109.0.5414
[WDM] - Get LATEST chromedriver version for 109.0.5414 google-chrome
[WDM] - Driver [C:\Users\krish\.wdm\drivers\chromedriver\win32\109.0.5414.74\chromedriver.exe] found in cache


In [106]:
article_links = driver.find_elements(By.CSS_SELECTOR, "div.row div.row h5>a[href]")
article_links = list(set([l.get_attribute('href') for l in article_links])) ## set <-> unique

In [107]:
len(article_links)

320

In [108]:
# empty list
headline_list = []
content_list = []

In [109]:
dates = driver.find_elements(By.CSS_SELECTOR, 'span.entry-date.pull-left.clearfix')
date_list = []

for date in dates:
    date_list.append(date.text)

In [110]:
for url in article_links:
    driver.get(url)
    ### SCRAPE ARTICLE ###
    # title
    headline = driver.find_element(By.TAG_NAME, "h1")
    headline_list.append(headline.text)
    # content
    content = driver.find_element(By.CLASS_NAME, "entry-content") 
    #content = ' '.join(content)
    content_list.append(content.text)

In [ ]:
#content_list = ' '.join(content)

In [111]:
df_test = pd.DataFrame(headline_list,columns=['headline'])

In [112]:
df_test['date'] = pd.Series(date_list)

In [113]:
df_test['content'] = pd.Series(content_list)

In [114]:
df_test['link'] = pd.Series(article_links)

In [115]:
df_test.head()

,headline,date,content,link
0,Sidak Duktang Polsek Dentim Nihil Pelanggaran,26 Jan 2023 08:37,"DENPASAR, NusaBali\nPersonel Polsek Denpasar t...",https://www.nusabali.com/berita/128473/sidak-d...
1,Disdukcapil Gencarkan Sosialisasi Identitas Ke...,24 Jan 2023 11:48,"DENPASAR, NusaBali\nDinas Kependudukan dan Pen...",https://www.nusabali.com/berita/133944/disdukc...
2,Menikah ke Ubud Langsung Dapat Akta Perkawinan,20 Jan 2023 11:24,"GIANYAR, NusaBali\nCamat Ubud Ir I Wayan Suwij...",https://www.nusabali.com/berita/111333/menikah...
3,Pemeriksaan KTP Masih Ditiadakan,20 Jan 2023 09:27,Dinas Dukcapil Jembrana bersama petugas gabung...,https://www.nusabali.com/berita/117181/pemerik...
4,"Seluruh Transaksi Pajak Pakai NIK, Berlaku Per...",19 Jan 2023 09:27,"JAKARTA, NusaBali\nNomor Induk Kependudukan (N...",https://www.nusabali.com/berita/121665/seluruh...


In [116]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   headline  320 non-null    object
 1   date      320 non-null    object
 2   content   320 non-null    object
 3   link      320 non-null    object
dtypes: object(4)
memory usage: 10.1+ KB


In [117]:
df_test.to_excel('kependudukan.xlsx')

In [25]:
(df_test.headline.values == 'Tekan').sum()

0